In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [3]:
# Load Data
data = pd.read_csv("processed_data.txt", sep="|")
data.head()

,Gender,Age_Scaled,Salary_Transformed_Scaled,Purchased
0,1,-1.781797,-1.809254,0
1,1,-0.253587,-1.752550,0
2,0,-1.113206,-0.725701,0
3,0,-1.017692,-0.246493,0
4,1,-1.781797,0.311281,0


In [7]:
# Define features
X = data[["Gender", "Age_Scaled", "Salary_Transformed_Scaled"]]

In [9]:
# Define target
y = data["Purchased"]

In [11]:
# Divide data into training and testing.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=35)

In [13]:
# Convert train and test data sets into tensors
import tensorflow as tf
tf.random.set_seed(35)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [15]:
# Define model and layers
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation="relu"), 
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [19]:
# Define model attributes
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)

In [21]:
# Train model
model.fit(X_train, y_train, batch_size=5, epochs=50, verbose=1)

Epoch 1/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6892 - loss: 0.6256
Epoch 2/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7257 - loss: 0.5470
Epoch 3/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7873 - loss: 0.4886
Epoch 4/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7799 - loss: 0.4460
Epoch 5/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7926 - loss: 0.4149
Epoch 6/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7965 - loss: 0.3917
Epoch 7/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8102 - loss: 0.3734
Epoch 8/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8192 - loss: 0.3579
Epoch 9/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8311 - loss: 0.3441
Epoch 10/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8435 - loss: 0.3318
Epoch 11/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8592 - loss: 0.3207
Epoch 12/50
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8631 - lo

In [23]:
# Evaluate model
results = model.evaluate(X_test, y_test, batch_size=5)
print("Model testing accuracy is", round(results[1], 3), "\nModel testing loss is", round(results[0], 3))

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9392 - loss: 0.1974  
Model testing accuracy is 0.933 
Model testing loss is 0.229


In [25]:
# Confusion Matrix
y_pred = model.predict(X_test)

tf.math.confusion_matrix(y_test, y_pred)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[36,  0],
       [24,  0]])>

In [27]:
# Import scaler object from "Preprocessing" notebook
%store -r scaler

In [94]:
# Function to make predictions from a set of input tensors
def predict_with_new_data(input):
    scaled_input = scaler.fit_transform(input)
    scaled_input_tensor = tf.convert_to_tensor(scaled_input, dtype=tf.float32)
    prediction = model.predict(tf.stack(scaled_input_tensor))
    return tf.round(prediction * 1000.0) / 1000.0

In [96]:
# Generate new, random data to test model
random_test_data = []

for i in np.arange(1,11):
    gender_random = np.random.choice([0, 1])
    age_random = int(np.random.uniform(18, 65))
    salary_transformed_random = int(np.random.uniform(180, 400))
    test_user_data = [gender_random, age_random, salary_transformed_random]
    random_test_data.append(test_user_data)

random_test_data

[[0, 21, 258],
 [0, 35, 317],
 [1, 56, 374],
 [1, 63, 210],
 [1, 23, 297],
 [1, 47, 286],
 [0, 29, 246],
 [0, 36, 312],
 [0, 60, 260],
 [0, 29, 331]]

In [98]:
predict_with_new_data(random_test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.   ],
       [0.04 ],
       [0.988],
       [0.99 ],
       [0.005],
       [0.192],
       [0.001],
       [0.031],
       [0.873],
       [0.045]], dtype=float32)>